<div align="center">
<p align="center" style="width: 100%;">
    <img src="https://raw.githubusercontent.com/vlm-run/.github/refs/heads/main/profile/assets/vlm-black.svg" alt="VLM Run Logo" width="80" style="margin-bottom: -5px; color: #2e3138; vertical-align: middle; padding-right: 5px;"><br>
</p>
<p align="center"><a href="https://docs.vlm.run"><b>Website</b></a> | <a href="https://docs.vlm.run/"><b>API Docs</b></a> | <a href="https://docs.vlm.run/blog"><b>Blog</b></a> | <a href="https://discord.gg/AMApC2UzVY"><b>Discord</b></a> | <a href="https://chat.vlm.run"><b>Chat</b></a>
</p>
</div>

# VLM Run Orion - Logic Gate Schematic to Truth Table

This notebook demonstrates how to use [VLM Run Orion's](https://vlm.run/orion) vision capabilities to analyze digital logic circuit diagrams and derive their Boolean expressions and complete truth tables. This is particularly useful for understanding complex logic circuits with multiple gates (AND, OR, XOR, NAND, NOR, NOT) and tracing signal paths from inputs to outputs.

For more details on the API, see the [Agent API docs](https://docs.vlm.run/agents/introduction).

## Prerequisites

- Python 3.10+
- VLM Run API key (get one at [app.vlm.run](https://app.vlm.run))
- VLM Run Python Client with OpenAI extra `vlmrun[openai]`


## Setup

First, install the required packages and configure the environment.


In [1]:
# Install required packages
%pip install vlmrun[openai] --upgrade --quiet
%pip install cachetools pillow requests numpy --quiet


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import getpass

VLMRUN_API_KEY = os.getenv("VLMRUN_API_KEY", None)
if VLMRUN_API_KEY is None:
    VLMRUN_API_KEY = getpass.getpass("Enter your VLM Run API key: ")


## Initialize the VLM Run Client

We use the OpenAI-compatible chat completions interface through the VLM Run SDK.


In [3]:
from vlmrun.client import VLMRun

BASE_URL = os.getenv("VLMRUN_BASE_URL", "https://agent.vlm.run/v1")
client = VLMRun(api_key=VLMRUN_API_KEY, base_url=BASE_URL)
print("VLM Run client initialized successfully!")
print(f"Base URL: {BASE_URL}")


VLM Run client initialized successfully!
Base URL: https://agent.vlm.run/v1


## Response Models

We define Pydantic models for structured outputs. The response will include the Boolean expression, truth table, intermediate expressions, and explanation.


In [4]:
from pydantic import BaseModel, Field
from typing import List, Dict, Any


class TruthTableResponse(BaseModel):
    """Response containing the Boolean expression and truth table for the logic circuit."""
    boolean_expression: str = Field(..., description="The final simplified Boolean algebraic expression for the entire circuit (e.g., Y = (A · B) + C̅)")
    truth_table: List[Dict[str, Any]] = Field(..., description="Complete truth table covering all possible input combinations, with columns for inputs, intermediate signals (if applicable), and final output")
    intermediate_expressions: List[str] = Field(default_factory=list, description="Boolean expressions for the output of each intermediate gate before reaching the final result")
    explanation: str = Field(..., description="Brief explanation of how the inputs interact to produce the final output state")

print("Response models defined successfully!")


Response models defined successfully!


## Helper Functions

We create helper functions to simplify making chat completion requests with structured outputs.


In [5]:
import hashlib
import json
from typing import Any, Type, TypeVar

import cachetools
from vlmrun.common.image import encode_image
from PIL import Image


T = TypeVar('T', bound=BaseModel)


def custom_key(prompt: str, image_path: str | None = None, response_model: Type[T] | None = None, model: str = "vlmrun-orion-1:auto"):
    """Custom key for caching chat_completion."""
    response_key = hashlib.sha256(json.dumps(response_model.model_json_schema(), sort_keys=True).encode()).hexdigest() if response_model else ""
    image_key = hashlib.sha256(image_path.encode()).hexdigest() if image_path else ""
    return (prompt, image_key, response_key, model)


@cachetools.cached(cache=cachetools.TTLCache(maxsize=100, ttl=3600), key=custom_key)
def chat_completion(
    prompt: str,
    image_path: str | None = None,
    response_model: Type[T] | None = None,
    model: str = "vlmrun-orion-1:auto"
) -> tuple[BaseModel | str, str]:
    """
    Make a chat completion request with structured output for logic circuit analysis.

    Args:
        prompt: The prompt describing the logic circuit analysis task
        image_path: Path to the image file containing the logic circuit diagram
        response_model: Pydantic model for structured output
        model: Model to use (default: vlmrun-orion-1:auto)

    Returns:
        Tuple of (parsed response model or text, session_id)
    """
    content = [{"type": "text", "text": prompt}]
    
    # Add image if provided
    if image_path:
        image = Image.open(image_path)
        image_data = encode_image(image, format="JPEG")
        content.append({"type": "image_url", "image_url": {"url": image_data}})

    kwargs = {
        "model": model,
        "messages": [{"role": "user", "content": content}]
    }

    if response_model:
        kwargs["response_format"] = {
            "type": "json_schema",
            "schema": response_model.model_json_schema()
        }

    response = client.agent.completions.create(**kwargs)
    response_text = response.choices[0].message.content
    session_id = response.session_id

    if response_model:
        result = response_model.model_validate_json(response_text)
        return result, session_id

    return response_text, session_id

print("Helper functions defined!")


Helper functions defined!


## Logic Gate Schematic to Truth Table Conversion

Analyze a digital logic circuit diagram to derive its Boolean expression and complete truth table. Provide the path to an image file containing the logic circuit schematic.


In [6]:
# Combined prompt for logic circuit analysis
LOGIC_CIRCUIT_PROMPT = """
Task: Analyze the attached digital logic circuit diagram to derive its Boolean expression and full truth table.

Instructions:

Circuit Decomposition: Identify every individual logic gate (e.g., AND, OR, NOT, XOR, NAND, NOR) and trace the signal paths from the inputs to the final output.

Intermediate Expressions: For complex circuits, please provide the Boolean expressions for the output of each intermediate gate before reaching the final result.

Final Boolean Expression: State the simplified Boolean algebraic expression for the entire circuit (e.g., $Y = (A \\cdot B) + \\overline{C}$).

Truth Table Generation:
- Construct a complete truth table covering all possible input combinations (e.g., $2^n$ rows for $n$ inputs).
- Include columns for intermediate signals if they help clarify the logic.
- Ensure the final output column is clearly labeled.

Verification: Briefly explain the logic of the circuit to confirm how the inputs interact to produce the final state.
"""

print("Logic circuit analysis prompt prepared!")
print(f"\nPrompt length: {len(LOGIC_CIRCUIT_PROMPT)} characters")


Logic circuit analysis prompt prepared!

Prompt length: 978 characters


In [7]:
# Example: Analyze logic circuit diagram
# Replace 'path/to/your/logic_circuit.jpg' with the actual path to your image
image_path = "dld.png"

# Generate the Boolean expression and truth table
result, session_id = chat_completion(
    prompt=LOGIC_CIRCUIT_PROMPT,
    image_path=image_path,
    response_model=TruthTableResponse,
    model="vlmrun-orion-1:auto"
)

print(">> RESPONSE")
print(result)
print(f"\n>> SESSION ID: {session_id}")

print("\n>> BOOLEAN EXPRESSION")
print("=" * 80)
print(result.boolean_expression)
print("=" * 80)

if result.intermediate_expressions:
    print("\n>> INTERMEDIATE EXPRESSIONS")
    print("=" * 80)
    for i, expr in enumerate(result.intermediate_expressions, 1):
        print(f"Gate {i}: {expr}")
    print("=" * 80)

print("\n>> TRUTH TABLE")
print("=" * 80)
# Print truth table in a formatted way
if result.truth_table:
    # Get column names from first row
    columns = list(result.truth_table[0].keys())
    # Print header
    header = " | ".join(str(col).ljust(12) for col in columns)
    print(header)
    print("-" * len(header))
    # Print rows
    for row in result.truth_table:
        row_str = " | ".join(str(row[col]).ljust(12) for col in columns)
        print(row_str)
print("=" * 80)

print("\n>> EXPLANATION")
print("=" * 80)
print(result.explanation)
print("=" * 80)


>> RESPONSE
boolean_expression='Q = (A \\cdot B) + (B \\cdot C) = B(A + C)' truth_table=[{}, {}, {}, {}, {}, {}, {}, {}] intermediate_expressions=['X1 = A \\cdot B (Output of the top AND gate)', 'X2 = B + C (Output of the middle OR gate)', 'X3 = B \\cdot C (Output of the bottom AND gate)', 'X4 = X2 \\cdot X3 = (B + C) \\cdot (B \\cdot C) = B \\cdot C (Output of the intermediate AND gate)'] explanation='The circuit determines its output based on the state of input B and the combined states of A and C. Specifically, the output Q is high (1) only when input B is high AND at least one of inputs A or C is high. The top path checks if A and B are both active, while the bottom logic path (the combination of OR and AND gates) effectively verifies if both B and C are active. The final OR gate outputs a 1 if either of these conditions is met.'

>> SESSION ID: 6bccb533-375d-480d-8e0a-924a58ceba03

>> BOOLEAN EXPRESSION
Q = (A \cdot B) + (B \cdot C) = B(A + C)

>> INTERMEDIATE EXPRESSIONS
Gate 1: 

---

## Conclusion

This notebook demonstrated how to use **VLM Run Orion** to analyze digital logic circuit diagrams and derive their Boolean expressions and complete truth tables.

### Key Takeaways

1. **Structured Prompts**: Well-structured prompts with clear instructions for circuit decomposition, intermediate expressions, and truth table generation produce better results.
2. **Session Management**: Use `session_id` to track your analysis requests.
3. **Image Input**: Provide clear, high-quality images of logic circuit diagrams for best results.
4. **Structured Output**: The model returns Boolean expressions, truth tables, intermediate expressions, and explanations in a structured format.

### Next Steps

- Experiment with different types of logic circuits (combinational, sequential, etc.)
- Try analyzing circuits with different gate types and complexities
- Explore the [VLM Run Documentation](https://docs.vlm.run) for more capabilities
- Join our [Discord community](https://discord.gg/AMApC2UzVY) for support

Happy analyzing! 🔌
